In [ ]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import requests
from furl import furl
from math import ceil
import json
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util
import sys
import pprint
import time

from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans, DBSCAN
from sklearn.preprocessing import StandardScaler

import seaborn as sns

from sklearn.metrics import silhouette_score

pp = pprint.PrettyPrinter()
%config InlineBackend.figure_format ='retina'

# My Library - K-Means Clustering

In [14]:
scores = []
for k in range(2, 31):
    
    # do the clustering
    km = KMeans(n_clusters=k, random_state=2020)
    km.fit(X)
    
    # get inertia
    inertia = km.inertia_
    
    # get silhouette
    sil = silhouette_score(X, km.labels_)
    
    # gather them
    scores.append((k, inertia, sil))

In [15]:
score_df = pd.DataFrame(scores, columns=['k', 'inertia', 'silhouette'])
score_df.sort_values('silhouette', ascending=False)

,k,inertia,silhouette
0,2,32630.014,0.174
2,4,28713.747,0.136
3,5,27070.325,0.134
4,6,25615.698,0.118
6,8,23363.176,0.117
7,9,22649.059,0.116
5,7,24335.211,0.115
8,10,21809.832,0.114
1,3,30542.463,0.111
10,12,20582.021,0.111


In [16]:
# (THREAD) How many K here?
# I do: Fit a k-means clustering model
km = KMeans(n_clusters=8, random_state=2020)
km.fit(X)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=8, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=2020, tol=0.0001, verbose=0)

In [17]:
km.cluster_centers_

array([[ 0.57639855,  0.28470368, -0.15721757,  0.36024936,  0.68702542,
        -0.22266219, -0.38994387, -0.33639313, -0.23251644,  0.5404832 ,
        -0.32458389, -0.03133344,  0.23128902],
       [-0.0723911 , -0.03814249, -0.02431839, -0.42002744, -0.06684378,
        -0.24858449,  0.09964568,  2.77883763, -0.16610909, -0.26159598,
        -0.08519559, -0.01943186,  0.14249526],
       [-0.84277851,  1.02783698, -0.16857108,  0.77485027,  0.39773251,
         0.0836379 , -0.83752677, -0.25528035,  0.02354785,  0.18721574,
         0.99518149, -0.24977527,  0.13570112],
       [-0.30239734,  0.66249334, -0.06567759,  0.47684235,  0.00704633,
         0.25816927, -0.54233174, -0.2402534 ,  2.87456072,  0.00381642,
         0.03899475,  0.0628725 ,  0.1243161 ],
       [-0.5939019 , -1.12215084, -0.03490622, -0.90280455,  0.08749738,
        -0.30562584,  0.96230845, -0.09569182, -0.22518937, -0.65281368,
        -0.02047027,  0.10932571, -2.66921971],
       [ 0.62807904,  0.130905

In [18]:
km.labels_

array([0, 0, 0, ..., 4, 4, 4], dtype=int32)

In [19]:
km.predict(X)

array([0, 0, 0, ..., 4, 4, 4], dtype=int32)

In [20]:
df['kmeans_label'] = km.labels_

In [21]:
df.head()

,track_id,title,artist,album,album_id,time_added,danceability,energy,key,loudness,...,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,kmeans_label
0,1GeId1N0DtmTpIfwr9aNQR,Lost in the Supermarket,The Clash,London Calling (Legacy Edition),3TZuLZx0zB3Q45aQXDswb9,['added_at'],0.610,0.594,4,-10.927,...,0.798,147.512,audio_features,1GeId1N0DtmTpIfwr9aNQR,spotify:track:1GeId1N0DtmTpIfwr9aNQR,https://api.spotify.com/v1/tracks/1GeId1N0DtmT...,https://api.spotify.com/v1/audio-analysis/1GeI...,227133,4,0
1,1d0RJmZXAncCnTYHFlhaj4,The Guns of Brixton,The Clash,London Calling (Legacy Edition),3TZuLZx0zB3Q45aQXDswb9,['added_at'],0.745,0.604,7,-9.311,...,0.648,96.514,audio_features,1d0RJmZXAncCnTYHFlhaj4,spotify:track:1d0RJmZXAncCnTYHFlhaj4,https://api.spotify.com/v1/tracks/1d0RJmZXAncC...,https://api.spotify.com/v1/audio-analysis/1d0R...,189667,4,0
2,6aLCZUnreez0cBOjKNeLDH,Spanish Bombs,The Clash,London Calling (Legacy Edition),3TZuLZx0zB3Q45aQXDswb9,['added_at'],0.616,0.776,9,-7.929,...,0.731,149.017,audio_features,6aLCZUnreez0cBOjKNeLDH,spotify:track:6aLCZUnreez0cBOjKNeLDH,https://api.spotify.com/v1/tracks/6aLCZUnreez0...,https://api.spotify.com/v1/audio-analysis/6aLC...,198867,4,0
3,124Y9LPRCAz3q2OP0iCvcJ,London Calling,The Clash,London Calling (Legacy Edition),3TZuLZx0zB3Q45aQXDswb9,['added_at'],0.654,0.799,0,-6.920,...,0.755,133.731,audio_features,124Y9LPRCAz3q2OP0iCvcJ,spotify:track:124Y9LPRCAz3q2OP0iCvcJ,https://api.spotify.com/v1/tracks/124Y9LPRCAz3...,https://api.spotify.com/v1/audio-analysis/124Y...,199733,4,0
4,4j4pPKE3xAblPIbhxScC1j,Rawnald Gregory Erickson the Second,STRFKR,Starfucker,4mBSeOEiQ4WgDaCnydb0tZ,['added_at'],0.900,0.538,7,-7.879,...,0.713,95.508,audio_features,4j4pPKE3xAblPIbhxScC1j,spotify:track:4j4pPKE3xAblPIbhxScC1j,https://api.spotify.com/v1/tracks/4j4pPKE3xAbl...,https://api.spotify.com/v1/audio-analysis/4j4p...,173693,4,1


In [22]:
df.groupby('kmeans_label').mean()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
kmeans_label,,,,,,,,,,,,,
0,0.608,0.651,4.784,-7.036,1.000,0.056,0.212,0.022,0.153,0.540,112.673,222494.128,3.987
1,0.505,0.566,5.263,-10.362,0.648,0.054,0.386,0.734,0.163,0.357,119.966,223513.748,3.948
2,0.383,0.847,4.743,-5.268,0.865,0.084,0.053,0.041,0.191,0.459,152.881,203780.000,3.945
3,0.468,0.751,5.114,-6.539,0.683,0.100,0.158,0.044,0.616,0.417,123.749,230564.844,3.940
4,0.422,0.281,5.225,-12.420,0.720,0.049,0.693,0.077,0.154,0.267,121.938,234544.537,2.706
5,0.616,0.611,5.593,-8.182,0.621,0.386,0.308,0.014,0.249,0.491,119.851,224360.869,3.924
6,0.574,0.673,6.705,-6.899,0.000,0.064,0.181,0.030,0.158,0.465,118.290,240514.188,3.985
7,0.508,0.270,5.450,-12.474,0.742,0.042,0.779,0.040,0.136,0.280,114.354,228749.424,4.023


In [23]:
df.groupby('kmeans_label').count()

,track_id,title,artist,album,album_id,time_added,danceability,energy,key,loudness,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
kmeans_label,,,,,,,,,,,,,,,,,,,,,
0,633,633,633,633,633,633,633,633,633,633,...,633,633,633,633,633,633,633,633,633,633
1,270,270,270,270,270,270,270,270,270,270,...,270,270,270,270,270,270,270,270,270,270
2,474,474,474,474,474,474,474,474,474,474,...,474,474,474,474,474,474,474,474,474,474
3,167,167,167,167,167,167,167,167,167,167,...,167,167,167,167,167,167,167,167,167,167
4,218,218,218,218,218,218,218,218,218,218,...,218,218,218,218,218,218,218,218,218,218
5,145,145,145,145,145,145,145,145,145,145,...,145,145,145,145,145,145,145,145,145,145
6,474,474,474,474,474,474,474,474,474,474,...,474,474,474,474,474,474,474,474,474,474
7,616,616,616,616,616,616,616,616,616,616,...,616,616,616,616,616,616,616,616,616,616


In [24]:
df[df['kmeans_label'] == 7].head(20)

,track_id,title,artist,album,album_id,time_added,danceability,energy,key,loudness,...,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,kmeans_label
5,35rf8iduzQ7vd8hFbDlv0o,Easy Easy,King Krule,6 Feet Beneath The Moon,6zmDdgPRKrvYwXk8eWv3iK,['added_at'],0.563,0.408,4,-8.972,...,0.234,146.908,audio_features,35rf8iduzQ7vd8hFbDlv0o,spotify:track:35rf8iduzQ7vd8hFbDlv0o,https://api.spotify.com/v1/tracks/35rf8iduzQ7v...,https://api.spotify.com/v1/audio-analysis/35rf...,169693,4,7
6,1Yxbxg5R6dQcYmOdjvlXP0,Roll The Bones - Audiotree Live Version,Shakey Graves,Shakey Graves on Audiotree Live (2013),3JsZjfjVa95IMuZ50eX7BU,['added_at'],0.597,0.549,1,-10.253,...,0.204,122.801,audio_features,1Yxbxg5R6dQcYmOdjvlXP0,spotify:track:1Yxbxg5R6dQcYmOdjvlXP0,https://api.spotify.com/v1/tracks/1Yxbxg5R6dQc...,https://api.spotify.com/v1/audio-analysis/1Yxb...,291596,4,7
10,67DoMEUfkR2EuI1wVFqkGQ,Modern Romance,Yeah Yeah Yeahs,Fever To Tell (Deluxe Remastered),44ePwTuWK88vnalqutqJEG,['added_at'],0.324,0.427,9,-9.108,...,0.039,137.299,audio_features,67DoMEUfkR2EuI1wVFqkGQ,spotify:track:67DoMEUfkR2EuI1wVFqkGQ,https://api.spotify.com/v1/tracks/67DoMEUfkR2E...,https://api.spotify.com/v1/audio-analysis/67Do...,191366,4,7
16,5piIwdQnqzbAv7wU7nNkQc,Poor Song,Yeah Yeah Yeahs,Fever To Tell (Deluxe Remastered),44ePwTuWK88vnalqutqJEG,['added_at'],0.555,0.146,11,-16.991,...,0.189,124.208,audio_features,5piIwdQnqzbAv7wU7nNkQc,spotify:track:5piIwdQnqzbAv7wU7nNkQc,https://api.spotify.com/v1/tracks/5piIwdQnqzbA...,https://api.spotify.com/v1/audio-analysis/5piI...,184126,4,7
20,0VbLcxrynVHddrt7sN9jsI,Blush,Wolf Alice,My Love Is Cool (Deluxe Edition),5Uqtbi6LDSKDNNzfVU58PM,['added_at'],0.328,0.439,9,-8.991,...,0.086,123.582,audio_features,0VbLcxrynVHddrt7sN9jsI,spotify:track:0VbLcxrynVHddrt7sN9jsI,https://api.spotify.com/v1/tracks/0VbLcxrynVHd...,https://api.spotify.com/v1/audio-analysis/0VbL...,259361,4,7
27,6wTZv8fxGUXrQdDy15iyaY,Breathless,I M U R,Little Death,6TpP63a6InKlHItjwf7wLp,['added_at'],0.677,0.281,10,-13.244,...,0.077,100.997,audio_features,6wTZv8fxGUXrQdDy15iyaY,spotify:track:6wTZv8fxGUXrQdDy15iyaY,https://api.spotify.com/v1/tracks/6wTZv8fxGUXr...,https://api.spotify.com/v1/audio-analysis/6wTZ...,252772,4,7
34,0XsbCr34semDxq0ZX84BCh,Easy Living,Billie Holiday,Billie Holiday Love Songs,3MXpFQDq1cTE4WfGZDAsY4,['added_at'],0.632,0.134,8,-17.029,...,0.618,89.088,audio_features,0XsbCr34semDxq0ZX84BCh,spotify:track:0XsbCr34semDxq0ZX84BCh,https://api.spotify.com/v1/tracks/0XsbCr34semD...,https://api.spotify.com/v1/audio-analysis/0Xsb...,180930,4,7
36,66jMYG0wj6m6oj8LKvteEn,So Afraid,Janelle Monáe,Dirty Computer,2PjlaxlMunGOUvcRzlTbtE,['added_at'],0.435,0.293,5,-10.850,...,0.099,79.983,audio_features,66jMYG0wj6m6oj8LKvteEn,spotify:track:66jMYG0wj6m6oj8LKvteEn,https://api.spotify.com/v1/tracks/66jMYG0wj6m6...,https://api.spotify.com/v1/audio-analysis/66jM...,243808,4,7
47,3vHZogGuw36HLk1BeGLhZq,Blues Run The Game,Laura Marling,Blues Run The Game,25Dqil85sRtH67ADs2GmrM,['added_at'],0.589,0.224,11,-11.502,...,0.536,93.313,audio_features,3vHZogGuw36HLk1BeGLhZq,spotify:track:3vHZogGuw36HLk1BeGLhZq,https://api.spotify.com/v1/tracks/3vHZogGuw36H...,https://api.spotify.com/v1/audio-analysis/3vHZ...,158147,4,7
49,1EYIDanK6mrRnFT9yYmfUk,Young In All The Wrong Ways,Sara Watkins,Young In All The Wrong Ways,5O41sFLk5rKN2S4eT8GgQR,['added_at'],0.466,0.463,9,-8.851,...,0.382,140.616,audio_features,1EYIDanK6mrRnFT9yYmfUk,spotify:track:1EYIDanK6mrRnFT9yYmfUk,https://api.spotify.com/v1/tracks/1EYIDanK6mrR...,https://api.spotify.com/v1/audio-analysis/1EYI...,258200,4,7


In [25]:
silhouette_score(X, km.labels_)

0.11681892506791094

In [26]:
df.head()

,track_id,title,artist,album,album_id,time_added,danceability,energy,key,loudness,...,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,kmeans_label
0,1GeId1N0DtmTpIfwr9aNQR,Lost in the Supermarket,The Clash,London Calling (Legacy Edition),3TZuLZx0zB3Q45aQXDswb9,['added_at'],0.610,0.594,4,-10.927,...,0.798,147.512,audio_features,1GeId1N0DtmTpIfwr9aNQR,spotify:track:1GeId1N0DtmTpIfwr9aNQR,https://api.spotify.com/v1/tracks/1GeId1N0DtmT...,https://api.spotify.com/v1/audio-analysis/1GeI...,227133,4,0
1,1d0RJmZXAncCnTYHFlhaj4,The Guns of Brixton,The Clash,London Calling (Legacy Edition),3TZuLZx0zB3Q45aQXDswb9,['added_at'],0.745,0.604,7,-9.311,...,0.648,96.514,audio_features,1d0RJmZXAncCnTYHFlhaj4,spotify:track:1d0RJmZXAncCnTYHFlhaj4,https://api.spotify.com/v1/tracks/1d0RJmZXAncC...,https://api.spotify.com/v1/audio-analysis/1d0R...,189667,4,0
2,6aLCZUnreez0cBOjKNeLDH,Spanish Bombs,The Clash,London Calling (Legacy Edition),3TZuLZx0zB3Q45aQXDswb9,['added_at'],0.616,0.776,9,-7.929,...,0.731,149.017,audio_features,6aLCZUnreez0cBOjKNeLDH,spotify:track:6aLCZUnreez0cBOjKNeLDH,https://api.spotify.com/v1/tracks/6aLCZUnreez0...,https://api.spotify.com/v1/audio-analysis/6aLC...,198867,4,0
3,124Y9LPRCAz3q2OP0iCvcJ,London Calling,The Clash,London Calling (Legacy Edition),3TZuLZx0zB3Q45aQXDswb9,['added_at'],0.654,0.799,0,-6.920,...,0.755,133.731,audio_features,124Y9LPRCAz3q2OP0iCvcJ,spotify:track:124Y9LPRCAz3q2OP0iCvcJ,https://api.spotify.com/v1/tracks/124Y9LPRCAz3...,https://api.spotify.com/v1/audio-analysis/124Y...,199733,4,0
4,4j4pPKE3xAblPIbhxScC1j,Rawnald Gregory Erickson the Second,STRFKR,Starfucker,4mBSeOEiQ4WgDaCnydb0tZ,['added_at'],0.900,0.538,7,-7.879,...,0.713,95.508,audio_features,4j4pPKE3xAblPIbhxScC1j,spotify:track:4j4pPKE3xAblPIbhxScC1j,https://api.spotify.com/v1/tracks/4j4pPKE3xAbl...,https://api.spotify.com/v1/audio-analysis/4j4p...,173693,4,1


In [44]:
# for i in range [0,8]:
# #    print('Cluster #' + i)
#     x = i
#     df[df['kmeans_label'] == x]['artist'].value_counts()

print('Cluster 0:')
print(df[df['kmeans_label'] == 0]['artist'].value_counts().head(20))
print()
print('Cluster 1:')
print(df[df['kmeans_label'] == 1]['artist'].value_counts().head(20))
print()
print('Cluster 2:')
print(df[df['kmeans_label'] == 2]['artist'].value_counts().head(20))
print()
print('Cluster 3:')
print(df[df['kmeans_label'] == 3]['artist'].value_counts().head(20))
print()
print('Cluster 4:')
print(df[df['kmeans_label'] == 4]['artist'].value_counts().head(20))
print()
print('Cluster 5:')
print(df[df['kmeans_label'] == 5]['artist'].value_counts().head(20))
print()
print('Cluster 6:')
print(df[df['kmeans_label'] == 6]['artist'].value_counts().head(20))
print()
print('Cluster 7:')
print(df[df['kmeans_label'] == 7]['artist'].value_counts().head(20))
print()

Cluster 0:
Pat The Bunny               16
Nicki Minaj                 15
Kesha                       15
Kanye West                  13
Violent Femmes              13
Hop Along                   13
Grimes                      10
The Avett Brothers          10
The Front Bottoms           10
Spice Girls                 10
Sylvan Esso                  9
The Tallest Man On Earth     9
Laura Marling                9
First Aid Kit                9
Young Wonder                 9
MARINA                       8
Rising Appalachia            8
Sixpence None The Richer     8
Rainbow Kitten Surprise      8
alt-J                        7
Name: artist, dtype: int64

Cluster 1:
José González                      34
Grimes                             21
Rising Appalachia                  15
Jawbreaker                          9
alt-J                               8
SALES                               7
Young Wonder                        7
Ides of Gemini                      7
The Civil Wars & T Bone Bu

---

# DBSCAN Clustering

In [28]:
X

array([[ 0.59136357,  0.0665977 , -0.37444212, ...,  0.81895869,
         0.02281408,  0.25990652],
       [ 1.44392039,  0.10457456,  0.45723009, ..., -0.85496308,
        -0.41451015,  0.25990652],
       [ 0.62925498,  0.75777661,  1.01167824, ...,  0.86835773,
        -0.30712258,  0.25990652],
       ...,
       [-0.49485699, -1.59678892, -0.92889027, ..., -0.333795  ,
         0.16538277, -2.00445348],
       [-1.20847863, -1.3803208 , -0.09721805, ..., -1.52836554,
         0.28864503, -6.53317347],
       [-1.23373957, -1.28158095,  1.01167824, ...,  2.56092311,
         0.49703529, -2.00445348]])

In [29]:
db = DBSCAN(eps=10, min_samples=2)
db.fit(X)
pd.Series(db.labels_).value_counts()

 0    2995
-1       2
dtype: int64

In [30]:
db = DBSCAN(eps=10, min_samples=2)
db.fit(X)
pd.Series(db.labels_).value_counts()

 0    2995
-1       2
dtype: int64

In [32]:
ss = StandardScaler()
X = ss.fit_transform(df[X_features])
db = DBSCAN(eps=1.8, min_samples=5)
db.fit(X)
pd.Series(db.labels_).value_counts().head(20)
df['dbscan_label'] = db.labels_

In [34]:
df[df['dbscan_label'] == 1].head()

,track_id,title,artist,album,album_id,time_added,danceability,energy,key,loudness,...,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,kmeans_label,dbscan_label
7,3XBPCbTU9rSGuyuK7Xbj4B,Sixteen Saltines,Jack White,Blunderbuss,6eSJ0lu0uwtiqXkP7Qrrno,['added_at'],0.603,0.868,11,-7.891,...,131.519,audio_features,3XBPCbTU9rSGuyuK7Xbj4B,spotify:track:3XBPCbTU9rSGuyuK7Xbj4B,https://api.spotify.com/v1/tracks/3XBPCbTU9rSG...,https://api.spotify.com/v1/audio-analysis/3XBP...,155720,4,6,1
16,5piIwdQnqzbAv7wU7nNkQc,Poor Song,Yeah Yeah Yeahs,Fever To Tell (Deluxe Remastered),44ePwTuWK88vnalqutqJEG,['added_at'],0.555,0.146,11,-16.991,...,124.208,audio_features,5piIwdQnqzbAv7wU7nNkQc,spotify:track:5piIwdQnqzbAv7wU7nNkQc,https://api.spotify.com/v1/tracks/5piIwdQnqzbA...,https://api.spotify.com/v1/audio-analysis/5piI...,184126,4,7,1
22,6d15UEB74JSsEL9frcH3UU,Paris,Little Dragon,Nabuma Rubberband,7cRFJj0IQqDZZ3Gm03aGbb,['added_at'],0.731,0.712,7,-6.938,...,76.503,audio_features,6d15UEB74JSsEL9frcH3UU,spotify:track:6d15UEB74JSsEL9frcH3UU,https://api.spotify.com/v1/tracks/6d15UEB74JSs...,https://api.spotify.com/v1/audio-analysis/6d15...,204040,4,6,1
25,3HqkmwPjEnEW6PXxbuonAK,Windows,Broken Bells,Meyrin Fields EP,74Duvq6GksnldghnjwYRdB,['added_at'],0.608,0.856,11,-5.338,...,97.002,audio_features,3HqkmwPjEnEW6PXxbuonAK,spotify:track:3HqkmwPjEnEW6PXxbuonAK,https://api.spotify.com/v1/tracks/3HqkmwPjEnEW...,https://api.spotify.com/v1/audio-analysis/3Hqk...,191987,4,6,1
26,4ESrHOeoOxvGSx9yLgM2aQ,Counting,Autre Ne Veut,Anxiety,0xOXSKdDGHiFGbi7Nf4CWT,['added_at'],0.618,0.538,2,-5.828,...,79.925,audio_features,4ESrHOeoOxvGSx9yLgM2aQ,spotify:track:4ESrHOeoOxvGSx9yLgM2aQ,https://api.spotify.com/v1/tracks/4ESrHOeoOxvG...,https://api.spotify.com/v1/audio-analysis/4ESr...,221840,4,6,1


In [37]:
df.groupby('dbscan_label').mean()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,kmeans_label
dbscan_label,,,,,,,,,,,,,,
-1,0.485,0.543,5.556,-10.045,0.562,0.130,0.399,0.257,0.259,0.390,124.445,233287.347,3.691,3.426
0,0.526,0.597,4.820,-8.005,1.000,0.060,0.331,0.036,0.167,0.440,122.122,218677.413,4.000,2.650
1,0.549,0.614,6.595,-7.710,0.000,0.057,0.284,0.026,0.158,0.426,120.958,229350.386,4.000,5.803
2,0.434,0.312,4.376,-10.442,1.000,0.043,0.675,0.011,0.126,0.245,127.903,228989.176,3.000,3.812
3,0.601,0.828,2.200,-6.496,1.000,0.062,0.040,0.744,0.304,0.329,99.707,238866.000,4.000,1.000
4,0.659,0.490,8.333,-11.472,0.000,0.034,0.820,0.827,0.107,0.454,117.429,183779.333,4.000,1.000
5,0.280,0.855,2.000,-6.087,1.000,0.042,0.005,0.622,0.185,0.524,128.497,322291.000,4.000,1.400
6,0.366,0.185,5.167,-12.073,0.000,0.039,0.836,0.016,0.116,0.167,127.573,208232.778,3.000,4.000
7,0.623,0.474,1.500,-9.473,1.000,0.046,0.043,0.815,0.162,0.423,125.795,269851.000,4.000,1.000


In [38]:
df.groupby('dbscan_label').count()

,track_id,title,artist,album,album_id,time_added,danceability,energy,key,loudness,...,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,kmeans_label
dbscan_label,,,,,,,,,,,,,,,,,,,,,
-1,779,779,779,779,779,779,779,779,779,779,...,779,779,779,779,779,779,779,779,779,779
0,1490,1490,1490,1490,1490,1490,1490,1490,1490,1490,...,1490,1490,1490,1490,1490,1490,1490,1490,1490,1490
1,588,588,588,588,588,588,588,588,588,588,...,588,588,588,588,588,588,588,588,588,588
2,85,85,85,85,85,85,85,85,85,85,...,85,85,85,85,85,85,85,85,85,85
3,5,5,5,5,5,5,5,5,5,5,...,5,5,5,5,5,5,5,5,5,5
4,6,6,6,6,6,6,6,6,6,6,...,6,6,6,6,6,6,6,6,6,6
5,5,5,5,5,5,5,5,5,5,5,...,5,5,5,5,5,5,5,5,5,5
6,18,18,18,18,18,18,18,18,18,18,...,18,18,18,18,18,18,18,18,18,18
7,6,6,6,6,6,6,6,6,6,6,...,6,6,6,6,6,6,6,6,6,6


In [41]:
df[df['dbscan_label'] == 3].head(20)

,track_id,title,artist,album,album_id,time_added,danceability,energy,key,loudness,...,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,kmeans_label,dbscan_label
48,2YvQY4h5KBhgF34YpNcz0z,Mouthful of Diamonds,Phantogram,Eyelid Movies,5aSsh4n7DFK3ngOCwa6ecp,['added_at'],0.613,0.815,0,-4.541,...,99.983,audio_features,2YvQY4h5KBhgF34YpNcz0z,spotify:track:2YvQY4h5KBhgF34YpNcz0z,https://api.spotify.com/v1/tracks/2YvQY4h5KBhg...,https://api.spotify.com/v1/audio-analysis/2YvQ...,253427,4,1,3
217,0s0tAkDH1NpMOErKA2pMo8,We out Here,Atomic Drum Assembly,Atomic Drum Assembly,5zQQA2PrCzkv3RbDaq32dL,['added_at'],0.636,0.842,2,-6.127,...,102.994,audio_features,0s0tAkDH1NpMOErKA2pMo8,spotify:track:0s0tAkDH1NpMOErKA2pMo8,https://api.spotify.com/v1/tracks/0s0tAkDH1NpM...,https://api.spotify.com/v1/audio-analysis/0s0t...,221120,4,1,3
831,7xab51UVPQHE0BPdPwzwS5,Bullet Proof,Morcheeba,Big Calm,7rjjFxMFZjPzQzBMj6mXr2,['added_at'],0.738,0.806,7,-6.436,...,94.066,audio_features,7xab51UVPQHE0BPdPwzwS5,spotify:track:7xab51UVPQHE0BPdPwzwS5,https://api.spotify.com/v1/tracks/7xab51UVPQHE...,https://api.spotify.com/v1/audio-analysis/7xab...,253093,4,1,3
1690,0VmPEJohARNJCfmZMzX8z5,Black Water - Chris Taylor Of Grizzly Bear Rem...,Of Monsters and Men,Beneath The Skin (Deluxe),1m3D8gkm3nOvIXNR1SvDFA,['added_at'],0.490,0.867,0,-7.976,...,115.504,audio_features,0VmPEJohARNJCfmZMzX8z5,spotify:track:0VmPEJohARNJCfmZMzX8z5,https://api.spotify.com/v1/tracks/0VmPEJohARNJ...,https://api.spotify.com/v1/audio-analysis/0VmP...,265760,4,1,3
2217,748S57nT4n56yhKbQrzQfh,Orange,Young Wonder,Young Wonder,0rENioYxtALAnrY5qQ6HjL,['added_at'],0.528,0.810,2,-7.400,...,85.987,audio_features,748S57nT4n56yhKbQrzQfh,spotify:track:748S57nT4n56yhKbQrzQfh,https://api.spotify.com/v1/tracks/748S57nT4n56...,https://api.spotify.com/v1/audio-analysis/748S...,200930,4,1,3


In [36]:
# scores = []

# for e in range(10, 30):
    
#     # do the clustering
#     db = DBSCAN(eps=(e/10), min_samples=5)
#     db.fit(X)
    
#     # get inertia

    
#     # get silhouette
#     sil = silhouette_score(X, db.labels_)
    
#     # gather them
#     scores.append(((e/10), inertia, sil))
    
#     print('with eps =' + str(e/10))
#     print(pd.Series(db.labels_).value_counts().head(20))